# 1.ETL

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('dataset.xlsx')

In [3]:
df.head()

,revenue*-/Line Id,Opportunity*Id,Line*Of*Business*Code,organization name,Territory*Name,Opportunity*Owner*Email,Customer****Name,Country,All*Competitors,Forecast*Type*Group,...,Executive*Product*Lob,Executive*Product*Class,Executive*Product*Pillar,Executive*Product*Line,Revenue*Line*Created*By,Revenue*Line*Probability,Best*Case**Pipeline*K,Worst*Case**Pipeline*K,Campaign*Type,Deal*Qualification
0,7123812,D434244A,HW/SW,LAD - SMB BDCs 1 - Divisional - ORCL Brazil,LAD_RRITA_OD,Melissa Perez,CUSTOMER-TDBNBY,BR,Other-Local,Pipeline,...,Cloud Infrastructure,PaaS,Universal Credits Workload,Move and Modernize ISV Apps (ISV customer),LUANA.LIMA@ORACLE.COM,10,50,0.0,NaN,NE
1,5857987,13C24C3A,Unassigned,LAD TECH License Sales 2 - ORCL Brazil,LAD_YZALEWSK,Leah Anderson,CUSTOMER-EGCAEV,BR,Unspecified,Upside,...,Cloud Infrastructure,PaaS,Universal Credits Workload,Other OCI Workloads,YURI.ZALEWSKY@ORACLE.COM,50,30,0.0,NaN,WEAK
2,5211345,DDA2C23C,HW/SW,FBGBU HW OD Others - ORCL Colombia,LAD_CAMIRODR,Jason Jordan,CUSTOMER-DVETKS,MX,Other-Local,Pipeline,...,Cloud Infrastructure,PaaS,Universal Credits Workload,OBSOLETE - HA/DR for Business Continuity,SANTIAGO.LOZANO@ORACLE.COM,10,12,0.0,NaN,NE
3,2227960,422BC2CD,HW/SW,GSO - Sales Central Operations - ORCL Scotland...,LAD_RICALOPE,Rebekah Munoz,CUSTOMER-GSASVP,MX,Unspecified,Won,...,License,On-Premise Technology,License - Database,Database,CRISTIANE.PESSOA@ORACLE.COM,100,64.3,64.3,NaN,WEAK
4,963452,34CC31DD,Unassigned,GSO - Sales Central Operations - ORCL Scotland...,LAD_DALZATE_,Adam Anderson,CUSTOMER-SIDXEE,CO,Other-Local,Pipeline,...,Cloud Infrastructure,PaaS,Universal Credits,PaaS and IaaS Universal Credits,ELIANA.G.GONZALEZ@ORACLE.COM,30,400,0.0,NaN,NE


In [4]:
print(f'Las columnas a trabajar son: {df.columns}')
print(f'El tamaño del dataset es: {df.shape}')

Las columnas a trabajar son: Index(['revenue*-/Line Id', 'Opportunity*Id', 'Line*Of*Business*Code',
       'organization   name', 'Territory*Name', 'Opportunity*Owner*Email',
       'Customer****Name', 'Country', 'All*Competitors', 'Forecast*Type*Group',
       'opportunity*--status', 'Oppty*Created*Date', 'Pipeline*K',
       'Opportunity*Close*Date', 'Fiscal*Quarter', 'Reason*Won*Lost',
       'Revenue*Line*Status', 'Opty*Forecast*Type', 'Revenue*Type*Group',
       'Industry*Name', 'Competitor', 'Revenue*Line*Creation*Date',
       'Executive*Product*Lob', 'Executive*Product*Class',
       'Executive*Product*Pillar', 'Executive*Product*Line',
       'Revenue*Line*Created*By', 'Revenue*Line*Probability',
       'Best*Case**Pipeline*K', 'Worst*Case**Pipeline*K', 'Campaign*Type',
       'Deal*Qualification'],
      dtype='object')
El tamaño del dataset es: (3955, 32)


In [5]:
df.describe()

,revenue*-/Line Id,Revenue*Line*Probability,Worst*Case**Pipeline*K,Campaign*Type
count,3.955000e+03,3955.000000,3955.000000,0.0
mean,4.987965e+06,40.257901,8.532441,NaN
std,2.899852e+06,31.169811,84.134562,NaN
min,5.990000e+02,10.000000,0.000000,NaN
25%,2.475241e+06,10.000000,0.000000,NaN
50%,4.944165e+06,30.000000,0.000000,NaN
75%,7.466541e+06,50.000000,0.000000,NaN
max,9.999609e+06,100.000000,4383.515000,NaN


In [6]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

organization   name     792
Campaign*Type          3955
Deal*Qualification      452
dtype: int64

Las columnas que presentan valores Nulos, son de tipo string por lo que no es necesario eliminarlas, se reemplaza los valores nulos por "No fill"

In [7]:
df.fillna('No fill', inplace=True)

In [8]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

Series([], dtype: float64)

In [9]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3955 entries, 0 to 3954
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   revenue*-/Line Id           3955 non-null   int64  
 1   Opportunity*Id              3955 non-null   object 
 2   Line*Of*Business*Code       3955 non-null   object 
 3   organization   name         3955 non-null   object 
 4   Territory*Name              3955 non-null   object 
 5   Opportunity*Owner*Email     3955 non-null   object 
 6   Customer****Name            3955 non-null   object 
 7   Country                     3955 non-null   object 
 8   All*Competitors             3955 non-null   object 
 9   Forecast*Type*Group         3955 non-null   object 
 10  opportunity*--status        3955 non-null   object 
 11  Oppty*Created*Date          3955 non-null   object 
 12  Pipeline*K                  3955 non-null   object 
 13  Opportunity*Close*Date      3955 

In [10]:
df.head()

,revenue*-/Line Id,Opportunity*Id,Line*Of*Business*Code,organization name,Territory*Name,Opportunity*Owner*Email,Customer****Name,Country,All*Competitors,Forecast*Type*Group,...,Executive*Product*Lob,Executive*Product*Class,Executive*Product*Pillar,Executive*Product*Line,Revenue*Line*Created*By,Revenue*Line*Probability,Best*Case**Pipeline*K,Worst*Case**Pipeline*K,Campaign*Type,Deal*Qualification
0,7123812,D434244A,HW/SW,LAD - SMB BDCs 1 - Divisional - ORCL Brazil,LAD_RRITA_OD,Melissa Perez,CUSTOMER-TDBNBY,BR,Other-Local,Pipeline,...,Cloud Infrastructure,PaaS,Universal Credits Workload,Move and Modernize ISV Apps (ISV customer),LUANA.LIMA@ORACLE.COM,10,50,0.0,No fill,NE
1,5857987,13C24C3A,Unassigned,LAD TECH License Sales 2 - ORCL Brazil,LAD_YZALEWSK,Leah Anderson,CUSTOMER-EGCAEV,BR,Unspecified,Upside,...,Cloud Infrastructure,PaaS,Universal Credits Workload,Other OCI Workloads,YURI.ZALEWSKY@ORACLE.COM,50,30,0.0,No fill,WEAK
2,5211345,DDA2C23C,HW/SW,FBGBU HW OD Others - ORCL Colombia,LAD_CAMIRODR,Jason Jordan,CUSTOMER-DVETKS,MX,Other-Local,Pipeline,...,Cloud Infrastructure,PaaS,Universal Credits Workload,OBSOLETE - HA/DR for Business Continuity,SANTIAGO.LOZANO@ORACLE.COM,10,12,0.0,No fill,NE
3,2227960,422BC2CD,HW/SW,GSO - Sales Central Operations - ORCL Scotland...,LAD_RICALOPE,Rebekah Munoz,CUSTOMER-GSASVP,MX,Unspecified,Won,...,License,On-Premise Technology,License - Database,Database,CRISTIANE.PESSOA@ORACLE.COM,100,64.3,64.3,No fill,WEAK
4,963452,34CC31DD,Unassigned,GSO - Sales Central Operations - ORCL Scotland...,LAD_DALZATE_,Adam Anderson,CUSTOMER-SIDXEE,CO,Other-Local,Pipeline,...,Cloud Infrastructure,PaaS,Universal Credits,PaaS and IaaS Universal Credits,ELIANA.G.GONZALEZ@ORACLE.COM,30,400,0.0,No fill,NE


In [11]:
no_name = df['organization   name'] == 'No fill'
df_organization_no_name = df[no_name]

In [12]:
no_name = df['organization   name'] == 'No fill'
print(no_name)

0       False
1       False
2       False
3       False
4       False
        ...  
3950    False
3951    False
3952    False
3953    False
3954    False
Name: organization   name, Length: 3955, dtype: bool


In [13]:
print(f'El total de registros que no cuentan con nombre de la empresa son {df_organization_no_name.shape}')
df_organization_no_name.head()

El total de registros que no cuentan con nombre de la empresa son (792, 32)


,revenue*-/Line Id,Opportunity*Id,Line*Of*Business*Code,organization name,Territory*Name,Opportunity*Owner*Email,Customer****Name,Country,All*Competitors,Forecast*Type*Group,...,Executive*Product*Lob,Executive*Product*Class,Executive*Product*Pillar,Executive*Product*Line,Revenue*Line*Created*By,Revenue*Line*Probability,Best*Case**Pipeline*K,Worst*Case**Pipeline*K,Campaign*Type,Deal*Qualification
5,584631,32CB44D1,HW/SW,No fill,LAD_ARBORGES,Mary Harvey,CUSTOMER-PFZRFM,BR,Unspecified,Upside,...,Cloud Infrastructure,PaaS,Universal Credits,PaaS and IaaS Universal Credits,FUSION-CRM-OIH-SPM_WW@ORACLE.COM,40,"1.,98404935670755",0.000000,No fill,No fill
8,5129333,D1D1CABC,HW/SW,No fill,LAD_EDELAV_T,Marc Lopez,CUSTOMER-WFKIZM,BR,Unspecified,Won,...,Cloud Infrastructure,PaaS,PaaS - Integration,PaaS - Integration,FUSION-CRM-CUST_WW@ORACLE.COM,100,8.624488,8.624488,No fill,WEAK
10,3857731,2431CDD3,Unassigned,No fill,LAD_RAMFRANC,Jessica Harris,CUSTOMER-SZQKJR,MX,Unspecified,Upside,...,Cloud Infrastructure,PaaS,Universal Credits,PaaS and IaaS Universal Credits,FUSION-CRM-CUST_WW@ORACLE.COM,50,12.089817,0.000000,No fill,WEAK
25,2280134,132341DC,HW/SW,No fill,LAD_DFOREROC,Mary Harvey,CUSTOMER-MZTEHW,MX,Unspecified,Won,...,Cloud Infrastructure,PaaS,Universal Credits,PaaS and IaaS Universal Credits,FUSION-CRM-OIH-SPM_WW@ORACLE.COM,100,15.022082,0.000000,No fill,No fill
36,3084231,422DBAB2,Unassigned,No fill,LAD_GNORTHCO,Christopher Haynes,CUSTOMER-UFXLJX,Peru,Unspecified,Won,...,Cloud Applications,Strategic Applications,Strategic Back Office,Fusion ERPM,FUSION-CRM-CUST_WW@ORACLE.COM,100,0,0.000000,No fill,No fill


In [14]:
df_organization_no_name['Opportunity*Owner*Email'].value_counts()

Mary Harvey       333
Alex Thomas         8
William Reid        7
Stacey Mathews      6
Robert Turner       6
                 ... 
Jared Valdez        1
Belinda White       1
Allen Owen          1
Tiffany Butler      1
Andrew Torres       1
Name: Opportunity*Owner*Email, Length: 315, dtype: int64

In [15]:
df['Opportunity*Owner*Email'].value_counts()

Mary Harvey          333
Christina Mcclure     15
Jay Mitchell          15
Michael Perez         14
Stacey Mathews        14
                    ... 
Michael Smith          1
Alexis Phillips        1
Jenny Rogers           1
Eric Cobb              1
Jason Bennett          1
Name: Opportunity*Owner*Email, Length: 1195, dtype: int64

In [16]:
no_dealq = df['Deal*Qualification'] == 'No fill'
df_no_dealq= df[no_dealq]

In [18]:
print(f'El total de registros que no cuentan con la calificacion de la propuesta son {df_no_dealq.shape}')
df_no_dealq.head()

El total de registros que no cuentan con la calificacion de la propuesta son (452, 32)


,revenue*-/Line Id,Opportunity*Id,Line*Of*Business*Code,organization name,Territory*Name,Opportunity*Owner*Email,Customer****Name,Country,All*Competitors,Forecast*Type*Group,...,Executive*Product*Lob,Executive*Product*Class,Executive*Product*Pillar,Executive*Product*Line,Revenue*Line*Created*By,Revenue*Line*Probability,Best*Case**Pipeline*K,Worst*Case**Pipeline*K,Campaign*Type,Deal*Qualification
5,584631,32CB44D1,HW/SW,No fill,LAD_ARBORGES,Mary Harvey,CUSTOMER-PFZRFM,BR,Unspecified,Upside,...,Cloud Infrastructure,PaaS,Universal Credits,PaaS and IaaS Universal Credits,FUSION-CRM-OIH-SPM_WW@ORACLE.COM,40,"1.,98404935670755",0.000000,No fill,No fill
25,2280134,132341DC,HW/SW,No fill,LAD_DFOREROC,Mary Harvey,CUSTOMER-MZTEHW,MX,Unspecified,Won,...,Cloud Infrastructure,PaaS,Universal Credits,PaaS and IaaS Universal Credits,FUSION-CRM-OIH-SPM_WW@ORACLE.COM,100,15.022082,0.000000,No fill,No fill
36,3084231,422DBAB2,Unassigned,No fill,LAD_GNORTHCO,Christopher Haynes,CUSTOMER-UFXLJX,Peru,Unspecified,Won,...,Cloud Applications,Strategic Applications,Strategic Back Office,Fusion ERPM,FUSION-CRM-CUST_WW@ORACLE.COM,100,0,0.000000,No fill,No fill
42,2600504,1C344BCD,Unassigned,No fill,LAD_ANGEALVA,Mary Harvey,customer-wtqlwk,MX,Unspecified,Upside,...,Cloud Infrastructure,PaaS,Universal Credits,PaaS and IaaS Universal Credits,FUSION-CRM-OIH-SPM_WW@ORACLE.COM,40,1290.327267,0.000000,No fill,No fill
50,3455642,322CA34A,Unassigned,LAD - SMB Tech Sales 1 - Divisional - ORCL Col...,LAD_JORGESAN,Christina Mcclure,customer-iyiesn,MX,Unspecified,Won,...,Cloud Infrastructure,PaaS,Universal Credits Workload,Move and Modernize ISV Apps (ISV customer),JORGE.S.SANCHEZ@ORACLE.COM,100,30.224442,30.224442,No fill,No fill


In [19]:
df_no_dealq['Opportunity*Owner*Email'].value_counts()

Mary Harvey           286
Randall Meyer           4
Mary Walton             4
Melissa Green           3
Michelle Davis          3
                     ... 
Eric Newman             1
Tristan Nelson DDS      1
Zoe Hanna               1
Joshua Gutierrez        1
Andrew Torres           1
Name: Opportunity*Owner*Email, Length: 134, dtype: int64